In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(rc={'figure.figsize': (10, 5)}, font_scale=1.5)

df = pd.read_csv('/content/drive/MyDrive/ECH_2019.csv')

# df = pd.read_csv('ECH_2019.csv')

In [3]:
no_data = {0: 'No hay dato', '0': 'No hay dato'}

mask = df.columns.difference(['edad',
                              'sueldo',
                              'hijos en hogar',
                              'hijos en otro hogar',
                              'hijos en el extranjero',
                              'suma_hijos',
                              'barrio'])

df[mask] = df[mask].replace(no_data)

In [4]:
df['rango_edades'] = pd.qcut(df['edad'], q=10)

df['rango_edades'].value_counts()

(8.0, 16.0]      11803
(39.0, 47.0]     11347
(63.0, 73.0]     10988
(-0.001, 8.0]    10918
(55.0, 63.0]     10848
(47.0, 55.0]     10834
(16.0, 23.0]     10657
(31.0, 39.0]     10484
(23.0, 31.0]     10116
(73.0, 98.0]      9876
Name: rango_edades, dtype: int64

### AJUSTANDO DATOS PARA ENTRENAR EL MODELO

Seleccionamos personas con edad suficiente para trabajar

In [5]:
df = df[~df['estado_laboral'].str.contains('Menores de 14 años')]

In [6]:
df['estado_laboral'].value_counts()

Ocupados                                      49036
Inactivo: jubilado                            16025
Inactivo: estudiante                           8216
Inactivo: realiza los quehaceres del hogar     5936
Inactivo: pensionista                          4438
Desocupados propiamente dichos                 3409
Inactivo: otro                                 1291
Desocupados buscan trabajo por primera vez      787
Inactivo: rentista                              414
Desocupados en seguro de paro                   255
Name: estado_laboral, dtype: int64

In [7]:
df.groupby(by='estado_laboral').mean()

,id_hogar,edad,nper,sueldo,barrio,hijos en hogar,hijos en otro hogar,hijos en el extranjero,suma_hijos
estado_laboral,,,,,,,,,
Desocupados buscan trabajo por primera vez,2.019029e+09,20.670902,3.035578,25.031766,11.876747,0.151207,0.022872,0.000000,0.174079
Desocupados en seguro de paro,2.019030e+09,39.074510,1.780392,3602.415686,12.407843,0.458824,0.254902,0.019608,0.733333
Desocupados propiamente dichos,2.019029e+09,33.915518,2.206512,1559.024641,11.319742,0.596656,0.310355,0.018774,0.925785
Inactivo: estudiante,2.019029e+09,17.345180,3.260346,93.646543,8.818768,0.033350,0.004260,0.000000,0.037610
Inactivo: jubilado,2.019029e+09,72.800936,1.348206,123.482059,10.446365,0.171108,0.959126,0.082683,1.212917
Inactivo: otro,2.019029e+09,33.783114,2.883036,618.094500,10.652982,0.237026,0.302866,0.023238,0.563129
Inactivo: pensionista,2.019029e+09,64.425192,1.683641,22.757999,9.698738,0.370662,1.675530,0.114015,2.160207
Inactivo: realiza los quehaceres del hogar,2.019029e+09,46.889319,1.857311,317.843666,8.732817,1.107311,1.102426,0.056941,2.266678
Inactivo: rentista,2.019029e+09,58.461353,1.528986,72.463768,9.009662,0.541063,0.850242,0.123188,1.514493


In [8]:
quantil_99 = df["sueldo"].quantile(0.99)
print(quantil_99)

df = df[df["sueldo"] < quantil_99]

quartil_1 = df["sueldo"].quantile(0.25)
quartil_3 = df["sueldo"].quantile(0.75)

iqr = quartil_3 - quartil_1

df = df[df["sueldo"] < quartil_3 + 1.5 * iqr]

df = df[df["sueldo"] > quartil_1 - 1.5 * iqr]

84000.0


In [9]:
df['rango_sueldos'] = pd.qcut(df['sueldo'], q=10, duplicates='drop').cat.codes

df['rango_edades'] = pd.qcut(df['edad'], q=10).cat.codes

In [10]:
df = df[[
    'sexo', 
    'edad', 
    'ascendencia', 
    'parentesco', 
    'estado_laboral', 
    'busca_otro_trabajo',
    'actividad',
    'asistencia alguna vez a enseñanza',
    'edu preescolar',
    'edu primaria',
    'edu media',
    'edu técnica',
    'edu univ o simil',
    'edu terciario no uni',
    'edu posgrado',
    'edu mag prof',
    'barrio',
    'estado_civil',
    'suma_hijos',
    'leer_escribir',
    'rango_sueldos',
    'rango_edades',
    'region',
    'vivienda',
    'estrato'
    ]]


In [11]:
df = df.reset_index(drop=True)

df.head(2)

,sexo,edad,ascendencia,parentesco,estado_laboral,busca_otro_trabajo,actividad,asistencia alguna vez a enseñanza,edu preescolar,edu primaria,edu media,edu técnica,edu univ o simil,edu terciario no uni,edu posgrado,edu mag prof,barrio,estado_civil,suma_hijos,leer_escribir,rango_sueldos,rango_edades,region,vivienda,estrato
0,Mujer,76.0,Blanca,Jefe/a de hogar,Inactivo: jubilado,0.0,.,Sí,No asistió,"Sí, asistió",0.0,No asistió,0.0,0.0,0.0,0.0,0.0,Viudo/a de unión libre,0.0,Sí,0,9,Interior - Localidades de 5.000 habitantes o más,Propietario de la vivienda y el terreno y ya l...,Costa Este
1,Mujer,22.0,Blanca,Otro pariente,Inactivo: realiza los quehaceres del hogar,0.0,.,Sí,"Sí, asistió","Sí, asistió","Sí, asistió",No asistió,No asistió,No asistió,0.0,No asistió,0.0,Soltero/a,0.0,Sí,0,1,Interior - Localidades de 5.000 habitantes o más,Propietario de la vivienda y el terreno y ya l...,Costa Este


In [12]:
df['rango_sueldos'].value_counts()

0    59982
1     9344
3     8546
2     7674
Name: rango_sueldos, dtype: int64

### DEFINIMOS LAS VARIABLES DEPENDIENTES E INDEPENDIENTES

In [13]:
# Feautures
X = df.drop('rango_sueldos', axis=1).to_numpy()
y = df['rango_sueldos']

print('X shape', X.shape, '\nY shape', y.shape)

X shape (85546, 24) 
Y shape (85546,)


In [14]:
y.unique()

array([0, 1, 3, 2], dtype=int8)

In [15]:
from sklearn.preprocessing import OneHotEncoder

one_hot = OneHotEncoder()

X = one_hot.fit_transform(X)

In [16]:
from sklearn.preprocessing import Normalizer

normalizer = Normalizer()

X = normalizer.fit_transform(X)

In [17]:
X.shape

(85546, 660)

In [18]:
test_sample = X[0]
test_result = y[0]

X = X[1:]
y = y[1:]

### Dividimos los datos en un 30% de test y 70% de entrenamiento

In [19]:
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42) 

In [20]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(59881, 660) (59881,)
(25664, 660) (25664,)


# DEFINIMOS ALGORITMO DE REGRESIÓN LOGISTICA

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

logistic_reg = LogisticRegression(multi_class='multinomial', max_iter=20000)

logistic_reg.fit(X_train, y_train)

LogisticRegression(max_iter=20000, multi_class='multinomial')

### VALOR DE CROSS VALIDATION PARA CADA FOLD

In [22]:
scores = cross_val_score(logistic_reg, X, y, cv=5)

scores

array([0.76339938, 0.76731545, 0.77017944, 0.76754924, 0.76620492])

In [23]:
print('Promedio de cross_validation ', round(scores.mean(),2))
print('Score en test ', round(logistic_reg.score(X_test, y_test),2))
print('Score en train ', round(logistic_reg.score(X_train, y_train),2))

Promedio de cross_validation  0.77
Score en test  0.77
Score en train  0.77


In [24]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

predictions = logistic_reg.predict(X_test)

confusion_matrix(y_test, predictions)

array([[17273,   381,   100,   351],
       [ 1550,   671,   244,   221],
       [  908,   357,   383,   652],
       [  777,   133,   240,  1423]])

### INVESTIGAMOS LA EFECTIVIDAD DEL MODELO SOBRE LOS DATOS

In [25]:
from sklearn.metrics import accuracy_score

#Prediccion en Train
y_train_pred = logistic_reg.predict(X_train)

#Prediccion en Test
y_test_pred = logistic_reg.predict(X_test)

#Calculo el accuracy en Train
train_accuracy = accuracy_score(y_train, y_train_pred)

#Calculo el accuracy en Test
test_accuracy = accuracy_score(y_test, y_test_pred)

print('Aciertos sobre entrenamiento:', round(train_accuracy, 2))
print('Aciertos sobre evaluación:', round(test_accuracy, 2))

Aciertos sobre entrenamiento: 0.77
Aciertos sobre evaluación: 0.77


### APLICAMOS PREDICCIÓN SOBRE EL MODELO

In [26]:
print(logistic_reg.predict(test_sample))

print(test_result)

[0]
0


Determinamos la probabilidad de la predicción

In [27]:
probabilidades_prediccion = logistic_reg.predict_proba(test_sample)
probabilidades_prediccion

array([[9.99027799e-01, 5.83432860e-04, 2.34311142e-04, 1.54456860e-04]])

# REPETIMOS EL EJERCICIO CON EL MODELO DE RANDOM FOREST

In [28]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(100)

random_forest.fit(X_train, y_train)

RandomForestClassifier()

In [29]:
scores = cross_val_score(random_forest, X, y, cv=5)

scores

array([0.75656087, 0.75170963, 0.75252791, 0.75258636, 0.75153428])

In [30]:
print('Promedio de cross_validation ', round(scores.mean(),2))
print('Score en test ', round(logistic_reg.score(X_test, y_test),2))
print('Score en train ', round(logistic_reg.score(X_train, y_train),2))

Promedio de cross_validation  0.75
Score en test  0.77
Score en train  0.77


In [31]:
predictions = logistic_reg.predict(X_test)

confusion_matrix(y_test, predictions)

array([[17273,   381,   100,   351],
       [ 1550,   671,   244,   221],
       [  908,   357,   383,   652],
       [  777,   133,   240,  1423]])

In [32]:
#Prediccion en Train
y_train_pred = random_forest.predict(X_train)

#Prediccion en Test
y_test_pred = random_forest.predict(X_test)

#Calculo el accuracy en Train
train_accuracy = accuracy_score(y_train, y_train_pred)

#Calculo el accuracy en Test
test_accuracy = accuracy_score(y_test, y_test_pred)

print('Aciertos sobre entrenamiento:', round(train_accuracy, 2))
print('Aciertos sobre evaluación:', round(test_accuracy, 2))

Aciertos sobre entrenamiento: 1.0
Aciertos sobre evaluación: 0.76


In [33]:
print(random_forest.predict(test_sample))
print(test_result)

[0]
0


In [34]:
probabilidades_prediccion = random_forest.predict_proba(test_sample)
probabilidades_prediccion

array([[1., 0., 0., 0.]])

# REPETIMOS EL EJERCICIO CON EL MODELO DE K NEAREST NEIGHBOURS

In [35]:
from sklearn.neighbors import KNeighborsClassifier

knearest_model = KNeighborsClassifier(n_neighbors=4)

knearest_model.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=4)

In [36]:
scores = cross_val_score(knearest_model, X, y, cv=5)

scores           

array([0.710211  , 0.70986031, 0.710211  , 0.71073704, 0.71015255])

In [37]:
print('Promedio de cross_validation ', round(scores.mean(),2))
print('Score en test ', round(knearest_model.score(X_test, y_test),2))
print('Score en train ', round(knearest_model.score(X_train, y_train),2))

Promedio de cross_validation  0.71
Score en test  0.71
Score en train  0.81


In [ ]:
predictions = knearest_model.predict(X_test)

confusion_matrix(y_test, predictions)

In [ ]:
#Prediccion en Train
y_train_pred = knearest_model.predict(X_train)

#Prediccion en Test
y_test_pred = knearest_model.predict(X_test)

#Calculo el accuracy en Train
train_accuracy = accuracy_score(y_train, y_train_pred)

#Calculo el accuracy en Test
test_accuracy = accuracy_score(y_test, y_test_pred)

print('Aciertos sobre entrenamiento:', round(train_accuracy, 2))
print('Aciertos sobre evaluación:', round(test_accuracy, 2))

In [ ]:
print(knearest_model.predict(test_sample))
print(test_result)

In [ ]:
probabilidades_prediccion = knearest_model.predict_proba(test_sample)
probabilidades_prediccion